# Multi-language processing
* we will deal with not only python, but also other languages. 
* and no script and notebook features.

# merge data tables

## for now we only have the voted kernels' results

In [1]:
import pandas as pd
import os

competition = r"J:\EECS_6414\Data\19-02-24\competition_split\competition_stats.csv"
datasets = r"J:\EECS_6414\Data\19-02-24\dataset_split\dataset_stats.csv"
kernel_measure_0 = r"J:\EECS_6414\Data\19-02-24\kernel_split\cleaning_result\measure_set_0\kernel_stats_combined.csv"
kernel_measure_1 = r"J:\EECS_6414\Data\19-02-24\kernel_split\cleaning_result\measure_set_1\kernel_stats.csv"

def change_column_names(table, table_identifier, exclude_columns = []):
    origin_col_name = list(table.columns.values)
    new_columns = {old_column: "{}_{}".format(table_identifier, old_column) for old_column in origin_col_name if old_column not in exclude_columns}
    table.rename(index=str, 
                    columns=new_columns, 
                    inplace= True)
    
def change_id_format(table):
    table["id"] = table["kernel"] + table["author"]
    table.drop(['kernel', 'author'], axis=1, inplace= True)

km0 = pd.read_csv(kernel_measure_0, encoding="utf-8")
km1 = pd.read_csv(kernel_measure_1, encoding="utf-8")
change_column_names(km0, "km", ["kernel", "author"])
change_column_names(km1, "km", ["kernel", "author"])
change_id_format(km0)
change_id_format(km1)
c = pd.merge(km1, km0, on= "id")
c.to_csv("kernel_datasource_first_merged.csv", index=None)

C:\Users\xmk233\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### calculate the high and low vote

In [2]:
import pandas as pd
import os

# cvo: code_voted_original
# cvp: code_voted_pyscript
# km: kernel_measures
merged_table = r"J:\EECS6414\process\Pipeline\Data_cleaning\kernel_datasource_first_merged.csv"
mt = pd.read_csv(merged_table, encoding="utf-8")

mt_voted = mt[mt["km_votes"] > 0]
print(mt_voted["km_votes"].describe())
print()
print(mt["km_votes"].describe())

count    41151.000000
mean        11.261160
std         58.748355
min          1.000000
25%          1.000000
50%          2.000000
75%          7.000000
max       3713.000000
Name: km_votes, dtype: float64

count    196969.000000
mean          2.352695
std          27.239849
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max        3713.000000
Name: km_votes, dtype: float64


C:\Users\xmk233\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
mt["high_voted"] = None
mt["high_voted"][mt["km_votes"] > 2] = True
mt["high_voted"][mt["km_votes"] <= 2] = False
mt["high_voted"].describe()

C:\Users\xmk233\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\xmk233\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count     196969
unique         2
top        False
freq      176731
Name: high_voted, dtype: object

### target encoding and some new features for tags

In [4]:
kernel_tags = {}
tags_kernel = {}
tags_high_and_low_vote_kernel_number = {}

how_many_kernels_dont_have_tags = 0
how_many_kernels_have_tags = 0

for index, row in mt.iterrows():
    kernel_id = row["id"]
    tag_string = row["km_tags"]
    if not isinstance(tag_string, str):
        how_many_kernels_dont_have_tags += 1
        tags_of_this_kernel = ["<(no_tags)>"]
    else:
        how_many_kernels_have_tags += 1
        tags_of_this_kernel = tag_string.split(";")
    for tag in tags_of_this_kernel:
        #####
        if tag in tags_kernel:
            tags_kernel[tag].append(kernel_id)
        else:
            tags_kernel[tag] = [kernel_id]
        #####
        if tag in tags_high_and_low_vote_kernel_number:
            if row["high_voted"]:
                tags_high_and_low_vote_kernel_number[tag][0] += 1
            else:
                tags_high_and_low_vote_kernel_number[tag][1] += 1
        else:
            tags_high_and_low_vote_kernel_number[tag] = [1, 0] if row["high_voted"] else [0, 1]
    kernel_tags[kernel_id] = tags_of_this_kernel

print(
    "how many kernels have tags: ", how_many_kernels_have_tags, "\n",
    "how_many_kernels_dont_have_tags: ", how_many_kernels_dont_have_tags, "\n",
    # "how many tags are in total: ", len(list(tags_num.keys())), "\n"
    tags_high_and_low_vote_kernel_number
)

def counting_tags(string):
    if isinstance(string, float):
        return 0
    return len(string.split(";"))

def target_encoding_tags_sum(string, return_stats="sum"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "sum":
        return sum(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_avg(string, return_stats="avg"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if (return_stats == "mean" or return_stats == "avg"):
        return mean(encoding_vals)
    else:
        return 0

def target_encoding_tags_max(string, return_stats="max"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "max":
        return max(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_min(string, return_stats="min"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "min":
        return min(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_median(string, return_stats="median"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "median":
        return median(encoding_vals)
    else:
        return 0

mt["km_num_of_tags"] = 0
mt["km_tags_te_sum"] = None
mt["km_tags_te_avg"] = None
mt["km_tags_te_max"] = None
mt["km_tags_te_min"] = None
mt["km_tags_te_median"] = None

mt["km_num_of_tags"] = mt["km_tags"].map(counting_tags)
mt["km_tags_te_sum"] = mt["km_tags"].map(target_encoding_tags_sum)
mt["km_tags_te_avg"] = mt["km_tags"].map(target_encoding_tags_avg)
mt["km_tags_te_max"] = mt["km_tags"].map(target_encoding_tags_max)
mt["km_tags_te_min"] = mt["km_tags"].map(target_encoding_tags_min)
mt["km_tags_te_median"] = mt["km_tags"].map(target_encoding_tags_median)

how many kernels have tags:  12288 
 how_many_kernels_dont_have_tags:  184681 
 {'eda': [1644, 801], 'data-visualization': [2726, 1666], 'feature-engineering': [817, 486], 'regression': [115, 107], 'starter-code': [709, 983], '<(no_tags)>': [13826, 170855], 'beginner': [2402, 2296], 'classification': [917, 931], 'time-series': [141, 65], 'twitter': [16, 14], 'tutorial': [1250, 793], 'nlp': [285, 140], 'text-mining': [122, 75], 'healthcare': [149, 44], 'health': [7, 5], 'gradient-boosting': [127, 53], 'binary-classification': [77, 84], 'linear-regression': [120, 168], 'data-cleaning': [586, 568], 'machine-learning': [29, 34], 'tabular-data': [24, 23], 'model-comparison': [85, 67], 'svm': [60, 75], 'multiclass-classification': [74, 61], 'image-processing': [122, 86], 'deep-learning': [519, 429], 'cnn': [203, 136], 'image-data': [106, 59], 'pipeline-code': [47, 23], 'neural-networks': [233, 188], 'optimization': [33, 12], 'naive-bayes': [40, 45], 'xgboost': [200, 100], 'dnn': [3, 6], 'clu

### target encoding and other measures for km_isNotebook

In [ ]:
tags_kernel = {}
tags_high_and_low_vote_kernel_number = {}

how_many_kernels_dont_have_tags = 0
how_many_kernels_have_tags = 0

for index, row in mt.iterrows():
    kernel_id = row["id"]
    tag = row["km_isNotebook"]
    if row["km_isNotebook"]:
        how_many_kernels_have_tags += 1
    else: 
        how_many_kernels_dont_have_tags += 1
    ##########
    if tag not in tags_kernel:
        tags_kernel[tag] = [kernel_id]
    else:
        tags_kernel[tag].append(kernel_id)
    ##########
    if tag in tags_high_and_low_vote_kernel_number:
        if row["high_voted"]:
            tags_high_and_low_vote_kernel_number[tag][0] += 1
        else:
            tags_high_and_low_vote_kernel_number[tag][1] += 1
    else:
        tags_high_and_low_vote_kernel_number[tag] = [1, 0] if row["high_voted"] else [0, 1]

print(
    "how many kernels are notebook: ", how_many_kernels_have_tags, "\n",
    "how_many_kernels_arent_notebook: ", how_many_kernels_dont_have_tags, "\n",
    # "how many tags are in total: ", len(list(tags_num.keys())), "\n"
    tags_high_and_low_vote_kernel_number
)

def target_encoding(tag):
    t = tags_high_and_low_vote_kernel_number
    return float(t[tag][0]) / sum(t[tag])

mt["km_isNotebook"] = mt["km_isNotebook"].map(target_encoding)

In [ ]:
tags_kernel = {}
tags_high_and_low_vote_kernel_number = {}

how_many_kernels_dont_have_tags = 0
how_many_kernels_have_tags = 0

for index, row in mt.iterrows():
    kernel_id = row["id"]
    tag = row["km_language"]
    ##########
    if tag not in tags_kernel:
        tags_kernel[tag] = [kernel_id]
    else:
        tags_kernel[tag].append(kernel_id)
    ##########
    if tag in tags_high_and_low_vote_kernel_number:
        if row["high_voted"]:
            tags_high_and_low_vote_kernel_number[tag][0] += 1
        else:
            tags_high_and_low_vote_kernel_number[tag][1] += 1
    else:
        tags_high_and_low_vote_kernel_number[tag] = [1, 0] if row["high_voted"] else [0, 1]

print(
    "how many kernels are notebook: ", how_many_kernels_have_tags, "\n",
    "how_many_kernels_arent_notebook: ", how_many_kernels_dont_have_tags, "\n",
    # "how many tags are in total: ", len(list(tags_num.keys())), "\n"
    tags_high_and_low_vote_kernel_number
)

def target_encoding(tag):
    t = tags_high_and_low_vote_kernel_number
    return float(t[tag][0]) / sum(t[tag])

mt["km_language"] = mt["km_language"].map(target_encoding)

In [ ]:
mt.to_csv("kernel_datassource.csv", index=None)

# Data stats